In [112]:
# Bibliotecas utilizadas


import requests
import urllib.request
import xml.etree.ElementTree as ET
import json
import pandas as pd

In [113]:
#coleta os elementos relacionados a uma coleção "samling"
r = requests.get('https://kulturarvsdata.se/ksamsok/api?method=getRelations&relation=all&objectId=SMVK-VKM/samling/2389&maxCount=10000')

In [115]:
df = pd.DataFrame()

root = ET.fromstring(r.text)
indiceobjeto=0
#itera pelos links das relações estabelecidas na coleção
for link in root.iter('relation'):  
    
    dicionario={} #resultados da coleta
    dicionario['id']=indiceobjeto
    # coleta o json de cada documento
    with urllib.request.urlopen("https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/"+link.text[41:]) as url:
        data = json.loads(url.read().decode())
        print ('Coletando o objeto :','https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/'+link.text[41:])
        for i in range(len(data['@graph'])):
            
            #print(data['@graph'][i])
            #print(data['@graph'][i]['@type'])
            # coleta os dados a partir de cada tipo
            
            #TIPO ItemDescription
            if data['@graph'][i]['@type']=='ItemDescription':
                metadado=data['@graph'][i]['type']['@value']
                valor=data['@graph'][i]['desc']['@value']
             #TIPO ItemName
            elif data['@graph'][i]['@type']=='ItemName':
                metadado=data['@graph'][i]['type']['@value']
                try: 
                    valor=data['@graph'][i]['name']['@value']
                except:
                    valor=data['@graph'][i]['http://kulturarvsdata.se/ksamsok#name']['@value']
            #TIPO ItemNumber
            elif data['@graph'][i]['@type']=='ItemNumber':
                metadado=data['@graph'][i]['type']['@value']
                valor=data['@graph'][i]['number']['@value']
            #TIPO Image
            elif data['@graph'][i]['@type']=='Image':
                metadado='Imagem'
                valor=data['@graph'][i]['lowresSource']
                
                #coletando arquivo da imagem
                try: 
                    response = requests.get(data['@graph'][i]['lowresSource'])
                    
                    #monta nome do arquivo
                    linkdoarquivo=data['@graph'][i]['lowresSource']

                    trechocodigo=linkdoarquivo[56:]
                    pos=trechocodigo.find("/")
                    nomearquivo=trechocodigo[:pos]
                    nomecompleto=str(indiceobjeto)+"-"+nomearquivo+".png"
                    
                    file = open(nomecompleto, "wb")
                    file.write(response.content)
                    file.close()
                    print("**** Coletando imagem: ",nomecompleto)
                except:
                    print('Imagem não disponível!')
            #TIPO Context
            elif data['@graph'][i]['@type']=='Context':
                #print(data['@graph'][i]['contextLabel']['@value'])
                if data['@graph'][i]['contextLabel']['@value']=='Ursprung': #origem
                    metadado='Ursprung'
                    valor=data['@graph'][i]['continentName']['@value']+' '+data['@graph'][i]['countryName']['@value']
                elif data['@graph'][i]['contextLabel']['@value']=='Insamlad': #coletado
                    metadado='Insamlad'
                    try: 
                        valor=data['@graph'][i]['foaf:name']['@value']+' - '+data['@graph'][i]['intermediary']
                    except:
                        valor=data['@graph'][i]['name']['@value']
                elif data['@graph'][i]['contextLabel']['@value']=='Brukad': #usado
                    metadado='Brukad'
                    try:
                        user=data['@graph'][i]['user']
                    except:
                        user=''
                    try:
                        valor=data['@graph'][i]['foaf:name']['@value']+' - '+user
                    except:
                        valor=data['@graph'][i]['name']['@value']+' - '+user
            #TIPO Entity
            elif data['@graph'][i]['@type']=='Entity':
                #XML
                URL=''
                palavrachave=''
                xmlpresentation = ET.fromstring(data['@graph'][i]['presentation'])
                for k in range(len(xmlpresentation)):
                    if 'description' in xmlpresentation[k].tag:
                        metadado='description'
                        valor=xmlpresentation[k].text
                    elif 'content' in xmlpresentation[k].tag:
                        metadado='content'
                        valor=xmlpresentation[k].text
                palavrachave=''#sempre existe para todos os objetos
                URL=data['@graph'][i]['url'] #sempre existe para todos os objetos
                for j in range(len(data['@graph'][i]['itemKeyWord'])):
                    if palavrachave:
                        try:
                            palavrachave=palavrachave+data['@graph'][i]['itemKeyWord'][j]['@value']
                        except:
                            palavrachave='NA' 
                    else:
                        try:
                            palavrachave=data['@graph'][i]['itemKeyWord'][j]['@value']
                        except:
                            palavrachave='NA'
            
            #ABSTRAINDO OS DADOS PARA MONTAR O DATAFRAME
            dicionario[metadado]=valor
            if data['@graph'][i]['@type']=='Entity':
                dicionario['Keywords']=palavrachave
                dicionario['URL']=URL

        #incrementa indice de objetos
        indiceobjeto=indiceobjeto+1  

    #Trabalha para guardar os dados no DATAFRAME
    df2 = pd.DataFrame(dicionario,index=[0])
    df = pd.concat([df, df2], axis=0)
                
    df.to_excel("output.xlsx")            

Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/14813
**** Coletando imagem:  0-1849972.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37812
**** Coletando imagem:  1-604709.png
**** Coletando imagem:  1-402290.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/14814
**** Coletando imagem:  2-2030928.png
**** Coletando imagem:  2-2030933.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37934
**** Coletando imagem:  3-1880174.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37811
**** Coletando imagem:  4-1209926.png
**** Coletando imagem:  4-604705.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37379
**** Coletando imagem:  5-1155425.png
**** Coletando imagem:  5-402154.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37930
**** Coletando imagem:  6-1880170.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VK

**** Coletando imagem:  37-15374285.png
**** Coletando imagem:  37-15374250.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37380
**** Coletando imagem:  38-600462.png
**** Coletando imagem:  38-1155429.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37381
**** Coletando imagem:  39-1155433.png
**** Coletando imagem:  39-600466.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/98996
**** Coletando imagem:  40-2536634.png
**** Coletando imagem:  40-2536644.png
**** Coletando imagem:  40-2536639.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37818
**** Coletando imagem:  41-604713.png
**** Coletando imagem:  41-1209938.png
**** Coletando imagem:  41-1209934.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37939
**** Coletando imagem:  42-600566.png
**** Coletando imagem:  42-1155525.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37819


**** Coletando imagem:  85-15374620.png
**** Coletando imagem:  85-15374600.png
**** Coletando imagem:  85-15374615.png
**** Coletando imagem:  85-15374610.png
**** Coletando imagem:  85-15374605.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37604
**** Coletando imagem:  86-1957740.png
**** Coletando imagem:  86-1957748.png
**** Coletando imagem:  86-15374630.png
**** Coletando imagem:  86-15374640.png
**** Coletando imagem:  86-15374655.png
**** Coletando imagem:  86-1957744.png
**** Coletando imagem:  86-15374650.png
**** Coletando imagem:  86-15374635.png
**** Coletando imagem:  86-15374645.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37601
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37722
**** Coletando imagem:  88-1049237.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37602
**** Coletando imagem:  89-2220299.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objek

Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/38140
**** Coletando imagem:  134-1155553.png
**** Coletando imagem:  134-600594.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/42770
**** Coletando imagem:  135-1957656.png
**** Coletando imagem:  135-1957644.png
**** Coletando imagem:  135-1957632.png
**** Coletando imagem:  135-1957648.png
**** Coletando imagem:  135-1957652.png
**** Coletando imagem:  135-1957640.png
**** Coletando imagem:  135-1957636.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37609
**** Coletando imagem:  136-1209842.png
**** Coletando imagem:  136-604641.png
**** Coletando imagem:  136-1209846.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37608
**** Coletando imagem:  137-1209834.png
**** Coletando imagem:  137-604637.png
**** Coletando imagem:  137-1209838.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37606
**** Coletando imagem: 

**** Coletando imagem:  175-600654.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/38164
**** Coletando imagem:  176-600658.png
**** Coletando imagem:  176-1155621.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/51140
**** Coletando imagem:  177-1880146.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/38161
**** Coletando imagem:  178-1155609.png
**** Coletando imagem:  178-600646.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/38162
**** Coletando imagem:  179-1155613.png
**** Coletando imagem:  179-600650.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/51138
**** Coletando imagem:  180-1320404.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/34136
**** Coletando imagem:  181-1988148.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37647
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/34

Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37658
**** Coletando imagem:  212-1222577.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37659
**** Coletando imagem:  213-15393625.png
**** Coletando imagem:  213-15393610.png
**** Coletando imagem:  213-15393620.png
**** Coletando imagem:  213-15393615.png
**** Coletando imagem:  213-15393585.png
**** Coletando imagem:  213-15393580.png
**** Coletando imagem:  213-15393600.png
**** Coletando imagem:  213-15393575.png
**** Coletando imagem:  213-15393605.png
**** Coletando imagem:  213-15393595.png
**** Coletando imagem:  213-15393590.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/38627
**** Coletando imagem:  214-15393545.png
**** Coletando imagem:  214-15393570.png
**** Coletando imagem:  214-15393555.png
**** Coletando imagem:  214-15393540.png
**** Coletando imagem:  214-15393560.png
**** Coletando imagem:  214-15393550.png
**** Coletando imagem:  214-1539356

**** Coletando imagem:  245-2768267.png
**** Coletando imagem:  245-2768262.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37407
**** Coletando imagem:  246-604625.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37649
**** Coletando imagem:  247-15393292.png
**** Coletando imagem:  247-15393282.png
**** Coletando imagem:  247-15393302.png
**** Coletando imagem:  247-15393272.png
**** Coletando imagem:  247-15393267.png
**** Coletando imagem:  247-15393287.png
**** Coletando imagem:  247-15393297.png
**** Coletando imagem:  247-15393277.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37408
**** Coletando imagem:  248-604629.png
**** Coletando imagem:  248-1209822.png
**** Coletando imagem:  248-1209818.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37548
**** Coletando imagem:  249-2220291.png
**** Coletando imagem:  249-1880106.png
Coletando o objeto : https://kulturarvsdata.se/SMV

Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37791
**** Coletando imagem:  282-402226.png
**** Coletando imagem:  282-604665.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37792
**** Coletando imagem:  283-402230.png
**** Coletando imagem:  283-604669.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37671
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/16866
**** Coletando imagem:  285-1334104.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/38659
**** Coletando imagem:  286-1049249.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/59699
**** Coletando imagem:  287-15375005.png
**** Coletando imagem:  287-15374995.png
**** Coletando imagem:  287-15374980.png
**** Coletando imagem:  287-15374975.png
**** Coletando imagem:  287-15374990.png
**** Coletando imagem:  287-15375000.png
**** Coletando imagem:  287-15374985.png
Coletando o objeto : 

**** Coletando imagem:  333-604773.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37903
**** Coletando imagem:  334-1155513.png
**** Coletando imagem:  334-600554.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37801
**** Coletando imagem:  335-1155457.png
**** Coletando imagem:  335-600486.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37923
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37802
**** Coletando imagem:  337-402258.png
**** Coletando imagem:  337-1155461.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37920
**** Coletando imagem:  338-1155521.png
**** Coletando imagem:  338-600562.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37921
**** Coletando imagem:  339-1225294.png
Coletando o objeto : https://kulturarvsdata.se/SMVK-VKM/objekt/jsonld/37800
**** Coletando imagem:  340-1155453.png
**** Coletando imagem:  340-60048

HTTPError: HTTP Error 404: Not Found